# Asignar valor de parámetro en el notebook desde un widget de texto


In [0]:
def assign_notebook_parameter_value_from_text_widget(widget_config: dict) -> None:
    """
    Crea widgets de texto en Databricks y asigna sus valores a variables de Python en el notebook.

    Esta función itera sobre un diccionario de widgets, creando cada widget de texto en Databricks y almacenando su
    valor en una variable de Python correspondiente al nombre del widget.

    Parámetros:
    - widget_config (dict): Un diccionario donde cada clave es el nombre del widget, y el valor es otro diccionario
                             que contiene 'value' (valor por defecto del widget) y 'label' (etiqueta del widget).
                             Ejemplo:
                             {
                                 'redWineUrl': {'value': 'http://example.com/redWine.csv', 'label': 'URL del Vino Tinto'},
                                 'saveDirectory': {'value': '/path/to/save', 'label': 'Directorio de Guardado' }
                             }

    Nota:
    - Esta función no devuelve ningún valor. Modifica los widgets de Databricks y asigna los valores de los widgets
      a las variables de Python correspondientes en el ámbito del notebook.
    """
    for widget_name, widget_info in widget_config.items():
        # Crear el widget de texto
        dbutils.widgets.text(widget_name, widget_info['value'], widget_info['label'])

    # Imprimir mensaje de éxito después de crear todos los widgets
    print("Proceso completado: Todos los widgets de texto han sido creados y asignados correctamente.")


def remove_assign_notebook_parameter_value_from_widget():
    """
    Elimina todos los widgets en el notebook de Databricks.
    """
    
    dbutils.widgets.removeAll()
    print("Todos los widgets han sido eliminados.")

In [0]:

# Configuración de widgets con valores y etiquetas
# Definir la URL base
url_base = 'https://raw.githubusercontent.com/JorgeCardona/data-collection-json-csv-sql/refs/heads/main/csv/wine'

# Definir la ruta común para el directorio de guardado
save_directory = '/FileStore/tables'  # Sin la barra final

# Definir los nombres de los datasets en formato snake_case
dataset_1_name = "winequality-red.csv"
dataset_2_name = "winequality-white.csv"

# Configuración de widgets con valores y etiquetas para los datasets
widget_config = {
    "save_directory": {"value": f"{save_directory}", "label": f"save_directory ({save_directory})"},
    "dataset_1_name": {"value": f"{dataset_1_name}", "label": f"Dataset name ({dataset_1_name})"},
    "dataset_2_name": {"value": f"{dataset_2_name}", "label": f"Dataset name ({dataset_2_name})"},
    "dataset1": {"value": f"{url_base}/{dataset_1_name}", "label": f"Dataset 1 URL ({dataset_1_name})"},
    "dataset2": {"value": f"{url_base}/{dataset_2_name}", "label": f"Dataset 2 URL ({dataset_2_name})"},
    "saveDirectory": {"value": save_directory, "label": "Save Directory"},
    "dataset1FileLocation": {"value": f"{save_directory}/{dataset_1_name}", "label": f"Dataset 1 File Location ({dataset_1_name})"},
    "dataset2FileLocation": {"value": f"{save_directory}/{dataset_2_name}", "label": f"Dataset 2 File Location ({dataset_2_name})"},
    "fileType": {"value": "csv", "label": "File Type"},
    "inferSchema": {"value": "true", "label": "Infer Schema"},
    "firstRowIsHeader": {"value": "true", "label": "First Row Is Header"},
    "delimiter": {"value": ",", "label": "Delimiter"},
    "bronze": {"value": "db_bronze", "label": "Data Base Bronze"},
    "silver": {"value": "db_silver", "label": "Data Base Silver"},
    "gold": {"value": "db_gold", "label": "Data Base Gold"},
    "dataset_1_table": {"value": "red_wine", "label": "Table Name Dataset 1"},
    "dataset_2_table": {"value": "white_wine", "label": "Table Name Dataset 2"}
}

# Llamar la función con el diccionario configurado
assign_notebook_parameter_value_from_text_widget(widget_config)

Proceso completado: Todos los widgets de texto han sido creados y asignados correctamente.


# FUNCIONES PARA DESCARGAR, LISTAR Y ELIMINAR ARCHIVOS EN UN DIRECTORIO

In [0]:
import requests
import os

# Función para descargar y guardar los archivos en databricks

def download_dataset_and_store_in_dbfs(url: str, save_path: str) -> None:
    """
    Descarga un archivo desde una URL y lo guarda en el Databricks File System (DBFS).

    Esta función descarga el contenido de un archivo desde una URL dada y lo guarda en una ruta dentro de DBFS.

    Parámetros:
    - url (str): URL desde donde se descargará el archivo.
    - save_path (str): Ruta completa en DBFS donde se guardará el archivo descargado.

    Excepciones:
    - requests.exceptions.RequestException: Si ocurre un error durante la descarga del archivo.
    - Exception: Para cualquier otro error inesperado, como problemas al guardar el archivo en DBFS.

    Ejemplo de uso:
    ```python
    download_dataset_and_store_in_dbfs("https://example.com/file.csv", "/dbfs/tmp/data/file.csv")
    ```

    El archivo será guardado en la ubicación especificada dentro de DBFS.
    """
    try:
        # Descargar el contenido del archivo
        response = requests.get(url)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa

        # Usar dbutils.fs.put para guardar el contenido en DBFS
        dbutils.fs.put(save_path, response.text, overwrite=True)

        print(f"Archivo guardado correctamente en {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el archivo desde {url}: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado al guardar el archivo en DBFS: {e}")


def list_files_in_directory_path(directory_path: str) -> None:
    """
    Lista los archivos dentro de un directorio especificado. Si no hay archivos, imprime un mensaje indicando
    que el directorio está vacío.

    Parámetros:
    - directory_path (str): Ruta completa del directorio cuyo contenido se desea listar.

    Retorno:
    - None: Esta función no retorna ningún valor, solo imprime el contenido del directorio o un mensaje de vacío.
    """
    files = dbutils.fs.ls(directory_path)
    if not files:
        print(f"No hay archivos en el directorio {directory_path}.")
    else:
        print(f"Archivos en el directorio {directory_path}:")
        for file in files:
            print(file.path)

def delete_all_files_in_directory_path(directory_path: str) -> None:
    """
    Elimina todos los archivos dentro de un directorio especificado. Si el directorio está vacío, imprime un mensaje 
    indicando que no hay archivos para eliminar.

    Parámetros:
    - directory_path (str): Ruta completa del directorio cuyo contenido se desea eliminar.

    Retorno:
    - None: Esta función no retorna ningún valor, solo imprime el estado de eliminación de los archivos.
    """
    # Listar todos los archivos en el directorio
    files = dbutils.fs.ls(directory_path)

    if not files:
        print(f"No hay archivos para eliminar en el directorio {directory_path}.\n")
    else:
        # Eliminar cada archivo encontrado
        print(f"Eliminando archivos en el directorio {directory_path}...\n")
        for file in files:
            dbutils.fs.rm(file.path, recurse=True)
            print(f"Archivo eliminado: {file.path}")
        
        print(f"\nTodos los archivos en el directorio {directory_path} han sido eliminados.\n")

# FUNCION PARA EJECUTAR LOS QUERIES EN SPARKSQL

In [0]:
def execute_spark_sql_query(query: str) -> "pyspark.sql.dataframe.DataFrame":
    """
    Ejecuta una consulta SQL en Spark y devuelve el resultado en un DataFrame.

    Parámetros:
    - query (str): Consulta SQL a ejecutar.

    Retorno:
    - pyspark.sql.dataframe.DataFrame: El DataFrame resultante de la ejecución de la consulta SQL.

    Nota:
    La función asume que la consulta SQL está correctamente formada y que se puede ejecutar en el entorno de Spark.
    """
    try:
        # Ejecutar el query SQL
        result_df = spark.sql(query)
        print(f"La consulta SQL se ejecutó exitosamente. El número de registros obtenidos es: {result_df.count()}")
        return result_df
    except Exception as e:
        print(f"Ocurrió un error al ejecutar la consulta SQL: {e}")
        return None

# FUNCIONES PARA GUARDAR LOS DATAFRAMES COMOL DELTA Y PARQUET

In [0]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

def standardize_column_names_from_spark_dataframe(spark_dataframe: "pyspark.sql.dataframe.DataFrame") -> "pyspark.sql.dataframe.DataFrame":
    """
    Estandariza los nombres de las columnas de un DataFrame de Spark reemplazando caracteres no alfanuméricos
    por guiones bajos ("_"). Esta transformación ayuda a evitar problemas de formato en el procesamiento de datos.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark con los nombres de columnas a estandarizar.

    Retorno:
    - pyspark.sql.dataframe.DataFrame: Nuevo DataFrame de Spark con los nombres de columnas estandarizados.
    """
    for col_name in spark_dataframe.columns:
        # Reemplazar caracteres no alfanuméricos por "_"
        new_col_name = ''.join([char if char.isalnum() or char == '_' else '_' for char in col_name])
        spark_dataframe = spark_dataframe.withColumnRenamed(col_name, new_col_name)
    
    print(f"Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.")
    return spark_dataframe


def save_dataframe_as_delta_format(spark_dataframe: "pyspark.sql.dataframe.DataFrame", path: str) -> None:
    """
    Guarda un DataFrame de Spark en formato Delta en una ubicación específica de almacenamiento.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar.
    - path (str): Ruta en el sistema de archivos donde se guardará el DataFrame en formato Delta.

    Retorno:
    - None

    Nota:
    Esta función sobrescribe cualquier archivo existente en la ruta especificada.
    """
    spark_dataframe = standardize_column_names_from_spark_dataframe(spark_dataframe)
    
    spark_dataframe.write.format("delta").mode("overwrite").save(path)
    print(f"El DataFrame ha sido guardado exitosamente en formato Delta en la ruta: {path}")


def save_dataframe_as_delta_table(spark_dataframe: "pyspark.sql.dataframe.DataFrame", database_name: str, table_name: str) -> None:
    """
    Guarda un DataFrame de Spark como una tabla Delta en una base de datos de Databricks.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar.
    - database_name (str): Nombre de la base de datos donde se creará la tabla.
    - table_name (str): Nombre de la tabla Delta a crear en la base de datos.

    Retorno:
    - None

    Nota:
    Esta función sobrescribe cualquier tabla existente con el mismo nombre en la base de datos especificada.
    """
    full_table_name = f"{database_name}.{table_name}"

    spark_dataframe = standardize_column_names_from_spark_dataframe(spark_dataframe)

    spark_dataframe.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
    print(f"El DataFrame ha sido guardado exitosamente como tabla Delta en la base de datos: {database_name}, tabla: {table_name}")


def save_dataframe_as_parquet_format(spark_dataframe: "pyspark.sql.dataframe.DataFrame", path: str) -> None:
    """
    Guarda un DataFrame de Spark en formato Parquet en una ubicación específica de almacenamiento.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar.
    - path (str): Ruta en el sistema de archivos donde se guardará el DataFrame en formato Parquet.

    Retorno:
    - None

    Nota:
    Esta función sobrescribe cualquier archivo existente en la ruta especificada.
    """
    spark_dataframe.write.format("parquet").mode("overwrite").save(path)
    print(f"El DataFrame ha sido guardado exitosamente en formato Parquet en la ruta: {path}")


def save_dataframe_as_parquet_table(spark_dataframe: "pyspark.sql.dataframe.DataFrame", database_name: str, table_name: str) -> None:
    """
    Guarda un DataFrame de Spark como una tabla Parquet en una base de datos de Databricks.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar.
    - database_name (str): Nombre de la base de datos donde se creará la tabla.
    - table_name (str): Nombre de la tabla Parquet a crear en la base de datos.

    Retorno:
    - None

    Nota:
    Esta función sobrescribe cualquier tabla existente con el mismo nombre en la base de datos especificada.
    """
    full_table_name = f"{database_name}.{table_name}"
    spark_dataframe.write.format("parquet").mode("overwrite").saveAsTable(full_table_name)
    print(f"El DataFrame ha sido guardado exitosamente como tabla Parquet en la base de datos: {database_name}, tabla: {table_name}")

def save_dataframe_as_temp_table(spark_dataframe: "pyspark.sql.dataframe.DataFrame", table_name: str) -> None:
    """
    Guarda un DataFrame de Spark como una tabla temporal en Databricks.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar como tabla temporal.
    - table_name (str): Nombre de la tabla temporal a crear.

    Retorno:
    - None

    Nota:
    Esta función crea o reemplaza una tabla temporal, que existe solo durante la sesión activa de Spark.
    """
    spark_dataframe.createOrReplaceTempView(table_name)
    print(f"El DataFrame ha sido guardado exitosamente como tabla temporal: {table_name}")


# FUNCION PARA LEER LOS ARCHIVOS DESCARGADOS COMO SPARK DATAFRAMES

In [0]:
from pyspark.sql import SparkSession

def load_data_to_spark_dataframe(file_location: str, file_type: str = "csv", infer_schema: bool = True, 
                           first_row_is_header: bool = True, delimiter: str = ",") -> "DataFrame":
    """
    Carga un archivo en un DataFrame de Spark, con opciones configurables para tipos de archivo, inferencia de esquema,
    encabezado, y delimitador.

    Parámetros:
    - file_location (str): La ubicación del archivo que se desea cargar.
    - file_type (str): El tipo de archivo. Por defecto es 'csv'. Otros valores pueden incluir 'parquet', 'json', etc.
    - infer_schema (bool): Si se debe inferir el esquema automáticamente del archivo. Por defecto es True.
    - first_row_is_header (bool): Si la primera fila debe ser usada como encabezado. Por defecto es True.
    - delimiter (str): El delimitador para los archivos CSV. Por defecto es ','.

    Retorna:
    - DataFrame: El DataFrame cargado con los datos del archivo.
    """
    try:
        # Cargar el archivo en el DataFrame
        df = spark.read.format(file_type) \
            .option("inferSchema", infer_schema) \
            .option("header", first_row_is_header) \
            .option("sep", delimiter) \
            .load(file_location)
        
        print(f"Archivo cargado correctamente desde {file_location}")
        return df
    
    except Exception as e:
        print(f"Error al cargar el archivo desde {file_location}: {e}")
        return None

# FUNCION PARA LISTAR LAS TABLAS DE LA BASE DE DATOS

In [0]:
def create_database(database_name: str) -> None:
    """
    Crea una base de datos en Databricks si no existe ya.

    Parámetros:
    - database_name (str): Nombre de la base de datos a crear.

    Retorno:
    - None

    Ejemplo de uso:
    ```python
    create_database("nombre_de_la_base_de_datos")
    ```
    """
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    print(f"La base de datos '{database_name}' ha sido creada exitosamente (o ya existía).")

def drop_database(database_name: str) -> None:
    """
    Elimina una base de datos en Databricks si existe.

    Parámetros:
    - database_name (str): Nombre de la base de datos que se desea eliminar.

    Retorno:
    - None

    Nota:
    Esta función elimina la base de datos especificada únicamente si existe. Para eliminar una base de datos que contenga
    tablas u objetos, utiliza la opción CASCADE dentro del comando SQL.

    Ejemplo de uso:
    ```python
    drop_database("nombre_de_la_base_de_datos")
    ```
    """
    spark.sql(f"DROP DATABASE IF EXISTS {database_name} CASCADE")
    print(f"La base de datos '{database_name}' ha sido eliminada exitosamente (si existía).")

def list_tables_in_database(database_name: str) -> None:
    """
    Lista y muestra todas las tablas en una base de datos específica en Databricks.

    Parámetros:
    - database_name (str): Nombre de la base de datos de la cual se quieren listar las tablas.

    Retorno:
    - None

    Ejemplo de uso:
    ```python
    list_tables_in_database("mi_base_de_datos")
    ```
    """
    tablas = spark.sql(f"SHOW TABLES IN {database_name}")
    display(tablas)

def list_temp_tables(show_global: bool = False) -> None:
    """
    Lista las tablas temporales en la sesión de Spark.

    Parámetros:
    - show_global (bool): Si es True, muestra tablas temporales globales (prefijadas con 'global_temp'). 
                          Si es False, muestra las tablas temporales locales de la sesión.

    Retorno:
    - None

    Nota:
    Esta función imprime las tablas temporales disponibles en la sesión actual de Spark.
    """
    if show_global:
        global_temp_tables = [table.name for table in spark.catalog.listTables("global_temp")]
        print("Tablas temporales globales:")
        for table in global_temp_tables:
            print(f"- global_temp.{table}")
    else:
        temp_tables = [table.name for table in spark.catalog.listTables() if table.isTemporary]
        print("Tablas temporales locales:")
        for table in temp_tables:
            print(f"- {table}")

## DESCARGAR LOS DATASETS ORIGINALES

In [0]:
# Recuperar los valores de los widgets
dataset1Url = dbutils.widgets.get("dataset1")
dataset2Url = dbutils.widgets.get("dataset2")
saveDirectory = dbutils.widgets.get("saveDirectory")
dataset_1_name = dbutils.widgets.get("dataset_1_name")
dataset_2_name = dbutils.widgets.get("dataset_2_name")

# Descargar y guardar los archivos de vino tinto y vino blanco en el almacenamiento local
download_dataset_and_store_in_dbfs(dataset1Url, os.path.join(saveDirectory, dataset_1_name))
download_dataset_and_store_in_dbfs(dataset2Url, os.path.join(saveDirectory, dataset_2_name))

# listar los archivos del directorio creado
list_files_in_directory_path(saveDirectory)

Wrote 100951 bytes.
Archivo guardado correctamente en /FileStore/tables/winequality-red.csv
Wrote 264426 bytes.
Archivo guardado correctamente en /FileStore/tables/winequality-white.csv
Archivos en el directorio /FileStore/tables:
dbfs:/FileStore/tables/winequality-red.csv
dbfs:/FileStore/tables/winequality-white.csv


## LEER LOS DATASETS DESCARGADOS COMO SPARK DATAFRAMES

In [0]:
# File locations and types
# Recuperar los valores de los widgets
fileType = dbutils.widgets.get("fileType")
inferSchema = dbutils.widgets.get("inferSchema")
firstRowIsHeader = dbutils.widgets.get("firstRowIsHeader")
delimiter = dbutils.widgets.get("delimiter")

dataset_1_name = dbutils.widgets.get("dataset_1_name")
dataset_2_name = dbutils.widgets.get("dataset_2_name")
saveDirectory = dbutils.widgets.get("saveDirectory")

dataset_1_location = f"{saveDirectory}/{dataset_1_name}"
dataset_2_location = f"{saveDirectory}/{dataset_2_name}"

red_wine_df = load_data_to_spark_dataframe(dataset_1_location, fileType, inferSchema, firstRowIsHeader, delimiter)
white_wine_df = load_data_to_spark_dataframe(dataset_2_location, fileType, inferSchema, firstRowIsHeader, delimiter)

display(red_wine_df.limit(5))
display(white_wine_df.limit(5))

Archivo cargado correctamente desde /FileStore/tables/winequality-red.csv
Archivo cargado correctamente desde /FileStore/tables/winequality-white.csv


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,6
7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6
6.3,0.3,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6
8.1,0.28,0.4,6.9,0.05,30.0,97.0,0.9951,3.26,0.44,10.1,6
7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.4,9.9,6
7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.4,9.9,6


### GUARDA EL DATAFRAME EN FORMATO DELTA

In [0]:
saveDirectory = dbutils.widgets.get("saveDirectory")

deltaPath1 = f"{saveDirectory}/Delta_dataset1"
deltaPath2 = f"{saveDirectory}/Delta_dataset2"

save_dataframe_as_delta_format(red_wine_df, deltaPath1)
print()
save_dataframe_as_delta_format(white_wine_df, deltaPath2)

print()
# listar los archivos del directorio creado
list_files_in_directory_path(deltaPath1)
print()
list_files_in_directory_path(deltaPath2)

Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
El DataFrame ha sido guardado exitosamente en formato Delta en la ruta: /FileStore/tables/Delta_dataset1

Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
El DataFrame ha sido guardado exitosamente en formato Delta en la ruta: /FileStore/tables/Delta_dataset2

Archivos en el directorio /FileStore/tables/Delta_dataset1:
dbfs:/FileStore/tables/Delta_dataset1/_delta_log/
dbfs:/FileStore/tables/Delta_dataset1/part-00000-2eb6b2da-a7b5-456b-b08e-fef1983726c2-c000.snappy.parquet

Archivos en el directorio /FileStore/tables/Delta_dataset2:
dbfs:/FileStore/tables/Delta_dataset2/_delta_log/
dbfs:/FileStore/tables/Delta_dataset2/part-00000-516c9ff8-d747-404f-81b3-efd5d5c484d1-c000.snappy.parquet


### GUARDA EL DATAFRAME EN FORMATO PARQUET

In [0]:
saveDirectory = dbutils.widgets.get("saveDirectory")

parquetPath1  = f"{saveDirectory}/Parquet_dataset1"
parquetPath2  = f"{saveDirectory}/Parquet_dataset2"

save_dataframe_as_parquet_format(red_wine_df, parquetPath1)
print()
save_dataframe_as_parquet_format(white_wine_df, parquetPath2)

print()
# listar los archivos del directorio creado
list_files_in_directory_path(parquetPath1)
print()
list_files_in_directory_path(parquetPath2)

El DataFrame ha sido guardado exitosamente en formato Parquet en la ruta: /FileStore/tables/Parquet_dataset1

El DataFrame ha sido guardado exitosamente en formato Parquet en la ruta: /FileStore/tables/Parquet_dataset2

Archivos en el directorio /FileStore/tables/Parquet_dataset1:
dbfs:/FileStore/tables/Parquet_dataset1/_SUCCESS
dbfs:/FileStore/tables/Parquet_dataset1/_committed_6795752328866930210
dbfs:/FileStore/tables/Parquet_dataset1/_started_6795752328866930210
dbfs:/FileStore/tables/Parquet_dataset1/part-00000-tid-6795752328866930210-652d195e-5da9-4ca5-8f5e-d50057712b95-443-1-c000.snappy.parquet

Archivos en el directorio /FileStore/tables/Parquet_dataset2:
dbfs:/FileStore/tables/Parquet_dataset2/_SUCCESS
dbfs:/FileStore/tables/Parquet_dataset2/_committed_7937310117741971714
dbfs:/FileStore/tables/Parquet_dataset2/_started_7937310117741971714
dbfs:/FileStore/tables/Parquet_dataset2/part-00000-tid-7937310117741971714-24078c0e-66af-48d1-8883-64d69b46fbbc-444-1-c000.snappy.parquet


### GUARDA EL DATAFRAME COMO TABLA EN FORMATO DELTA

In [0]:
database = dbutils.widgets.get("bronze")
create_database(database)

La base de datos 'db_bronze' ha sido creada exitosamente (o ya existía).


In [0]:
saveDirectory = dbutils.widgets.get("saveDirectory")
database = dbutils.widgets.get("bronze")
tabla_dataset1 = f'dataset1_delta'
tableParquetPath1  = f"{saveDirectory}/{tabla_dataset1}"

tabla_dataset2 = f'dataset2_delta'
tableParquetPath2  = f"{saveDirectory}/{tabla_dataset2}"

save_dataframe_as_delta_table(red_wine_df, database, tabla_dataset1)
print()
save_dataframe_as_delta_table(white_wine_df, database, tabla_dataset2)

print()
list_tables_in_database(database)

Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
El DataFrame ha sido guardado exitosamente como tabla Delta en la base de datos: db_bronze, tabla: dataset1_delta

Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
El DataFrame ha sido guardado exitosamente como tabla Delta en la base de datos: db_bronze, tabla: dataset2_delta



database,tableName,isTemporary
db_bronze,dataset1_delta,false
db_bronze,dataset2_delta,false


### GUARDA EL DATAFRAME COMO TABLA EN FORMATO PARQUET

In [0]:
saveDirectory = dbutils.widgets.get("saveDirectory")
database = dbutils.widgets.get("bronze")
tabla_dataset1 = f'dataset1_parquet'
tableParquetPath1  = f"{saveDirectory}/{tabla_dataset1}"

tabla_dataset2 = f'dataset2_parquet'
tableParquetPath2  = f"{saveDirectory}/{tabla_dataset2}"

save_dataframe_as_parquet_table(red_wine_df, database, tabla_dataset1)
print()
save_dataframe_as_parquet_table(white_wine_df, database, tabla_dataset2)

print()
list_tables_in_database(database)

El DataFrame ha sido guardado exitosamente como tabla Parquet en la base de datos: db_bronze, tabla: dataset1_parquet

El DataFrame ha sido guardado exitosamente como tabla Parquet en la base de datos: db_bronze, tabla: dataset2_parquet



database,tableName,isTemporary
db_bronze,dataset1_delta,false
db_bronze,dataset1_parquet,false
db_bronze,dataset2_delta,false
db_bronze,dataset2_parquet,false


## GUARDAR EL DATAFRAME COMO TABLAS TEMPORALES

In [0]:
save_dataframe_as_temp_table

saveDirectory = dbutils.widgets.get("saveDirectory")
tabla_dataset1 = f'dataset1_temp'
tableParquetPath1  = f"{saveDirectory}/{tabla_dataset1}"

tabla_dataset2 = f'dataset2_temp'
tableParquetPath2  = f"{saveDirectory}/{tabla_dataset2}"

save_dataframe_as_temp_table(red_wine_df, tabla_dataset1)
print()
save_dataframe_as_temp_table(white_wine_df, tabla_dataset2)

print()

# Ejemplo de uso:
# Listar tablas temporales locales
list_temp_tables()

print()

# Listar tablas temporales globales
list_temp_tables(show_global=True)

El DataFrame ha sido guardado exitosamente como tabla temporal: dataset1_temp

El DataFrame ha sido guardado exitosamente como tabla temporal: dataset2_temp

Tablas temporales locales:
- dataset1_temp
- dataset2_temp

Tablas temporales globales:
- global_temp.dataset1_temp
- global_temp.dataset2_temp


# ELIMINAS LOS PARAMETROS DEL NOTEBOOK Y LOS ARCHIVOS CREADOS

In [0]:
database = dbutils.widgets.get("bronze")
drop_database(database)

delete_all_files_in_directory_path(saveDirectory)
remove_assign_notebook_parameter_value_from_widget()

La base de datos 'db_bronze' ha sido eliminada exitosamente (si existía).
Eliminando archivos en el directorio /FileStore/tables...

Archivo eliminado: dbfs:/FileStore/tables/Delta_dataset1/
Archivo eliminado: dbfs:/FileStore/tables/Delta_dataset2/
Archivo eliminado: dbfs:/FileStore/tables/Parquet_dataset1/
Archivo eliminado: dbfs:/FileStore/tables/Parquet_dataset2/
Archivo eliminado: dbfs:/FileStore/tables/winequality-red.csv
Archivo eliminado: dbfs:/FileStore/tables/winequality-white.csv

Todos los archivos en el directorio /FileStore/tables han sido eliminados.

Todos los widgets han sido eliminados.
